In [180]:
import sys
sys.path.append('/home/moetjens/kidd-lab/jmkidd-projects/scripts/nextgen')
sys.path.append('/home/moetjens/kidd-lab/jmkidd-projects/scripts/pythonUtils')
import genutils
from optparse import OptionParser
import fastqstats
from collections import defaultdict

In [181]:
#arguements for inputs, outputs, and sample name

USAGE = """
Program on the command line here
"""

parser = OptionParser()
parser.add_option('--r1fq',dest='r1fq', help = 'name of f1 fq.gz')
parser.add_option('--r2fq',dest='r2fq', help = 'name of f2 fq.gz')
parser.add_option('--sample',dest='sampleName', help = 'name of sample')

#(options, args) = parser.parse_args()

#if options.r1fq is None:
#    parser.error('r1fq name not given')

#if options.r2fq is None:
#    parser.error('r2fq not given')

#if options.sampleName is None:
#    parser.error('sampleName not given')

r1fqFile= "/home/moetjens/kidd-lab-scratch/moetjens-projects/bsmn/MIPs/Run_2015-03-06/Trp53MIP_S19_L001_R1_001.fastq.gz"
r2fqFile= "/home/moetjens/kidd-lab-scratch/moetjens-projects/bsmn/MIPs/Run_2015-03-06/Trp53MIP_S19_L001_R2_001.fastq.gz"




In [182]:
#set up myData dictionary
# setup file location info
myData = {}
#myData['r1fq'] = options.r1fq
#myData['r2fq'] = options.r2fq
#myData['sampleName'] = options.sampleName
myData['r1fq'] = r1fqFile
myData['r2fq'] = r2fqFile
myData['sampleName'] = "Trp53"
myData['tags'] = defaultdict(int) #tag will be the key and the family size will be the count
barcode_dict = defaultdict(int)

In [183]:
#iterate through the fastq files

def fastq_iterator(myData):
    #count =0
    fqfileR1 = genutils.open_gzip_read(myData['r1fq'])
    fqfileR2 = genutils.open_gzip_read(myData['r2fq'])
    while True: 
        #count+=1
        R1 = fastqstats.get_next_seq_record(fqfileR1)
        R2 = fastqstats.get_next_seq_record(fqfileR2)
        if R1 is None: break 
        if R2 is None: break
        yield (R1['seq'], R2['seq'])
        #if(count>1000): break

readCount = 0
for read1Seq, read2Seq in fastq_iterator(myData):
    readCount +=1
    tag = read2Seq[:11]
    if(tag not in barcode_dict):
        barcode_dict.update({tag:int(1)})
    else:
        barcode_dict[tag] += int(1)

print("%s %s" % ("total number of reads is",str(readCount)))
    


total number of reads is 2356117


In [184]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import numpy as np
import pandas as pd
import scipy.misc

In [185]:
barcodeCounts= pd.DataFrame({'counts': barcode_dict.values()})
print barcodeCounts['counts'].describe()

file = open('/home/moetjens/kidd-lab-scratch/moetjens-projects/bsmn/MIPs/Run_2015-03-06/barcodeCounts.txt','w')

for key, value in barcode_dict.iteritems():
    file.write("%s\t%s\n" % (key,value))

file.close()


#histogram w/ Pandas
#myTicksX = range(0, 1200, 100)
#myLimX = [0,800] 
#len(barcodeCounts)
#barcodeCounts[1:100]
#merged.plot(kind='hist', stacked=True, bins=20)

count    1401129.000000
mean           1.681585
std           87.741409
min            1.000000
25%            1.000000
50%            1.000000
75%            2.000000
max       102716.000000
dtype: float64
